In [1]:
# import necessary packages
import sys, os
import torch 
import numpy as np
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from trl import SFTTrainer, setup_chat_format
from transformers import (pipeline,
                          AutoTokenizer,
                          AutoModelForCausalLM,
                          DataCollatorWithPadding,
                          get_scheduler)
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
from importlib import reload
from functools import partial
from IPython.display import clear_output

sys.path.append('../')

# custom imports
from utils.GetLowestGPU import GetLowestGPU
from utils.GetFileNames import get_file_names
import utils.preprocessing as pp

[2024-09-09 19:38:21,226] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -lcufile
collect2: error: ld returned 1 exit status


Device set to cuda:0


In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Instantiate Model and Dataset

In [3]:
# options
model_path = "meta-llama/Meta-Llama-3-8B-Instruct"
dataset_path = "ruslanmv/ai-medical-chatbot" #test dataset

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

# load tokenizer and model
pipeline = pipeline(
    "text-generation",
    model=model_path,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

pipeline.model = get_peft_model(pipeline.model, peft_config)

pipeline.tokenizer.pad_token = pipeline.tokenizer.eos_token
pipeline.tokenizer.pad_token_id = pipeline.tokenizer.eos_token_id
pipeline.model.generation_config.pad_token_id = pipeline.tokenizer.eos_token_id

# pipeline.model, pipeline.tokenizer = setup_chat_format(pipeline.model, pipeline.tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
pipeline.model.print_trainable_parameters()

trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.0424


In [5]:
# load dataset
raw_dataset = load_dataset(dataset_path, split = "train[:5000]")

# check format of data
raw_dataset = raw_dataset.train_test_split(test_size=0.1)
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['Description', 'Patient', 'Doctor'],
        num_rows: 4500
    })
    test: Dataset({
        features: ['Description', 'Patient', 'Doctor'],
        num_rows: 500
    })
})

# Preprocessing

In [6]:
# format chat dataset
reload(pp)
format_chat = partial(pp.format_chat, input_col="Patient", output_col="Doctor", pipeline_name=pipeline)
chat_dataset = raw_dataset.map(format_chat)
chat_dataset['test']

Parameter 'function'=functools.partial(<function format_chat at 0x7f6e879cf790>, input_col='Patient', output_col='Doctor', pipeline_name=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7f6e829aa220>) of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['Description', 'Patient', 'Doctor', 'input', 'output'],
    num_rows: 500
})

In [7]:
# tokenize dataset
reload(pp)
tokenize_function = partial(pp.tokenize_data, pipeline_name=pipeline, type="chat")
tokenized_dataset = chat_dataset.map(tokenize_function, 
                                     batched=True,
                                     remove_columns=chat_dataset['train'].column_names)

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
print("inputs:", tokenized_dataset['test'][0]['input_ids'][:10])
print("labels:", tokenized_dataset['test'][0]['labels'][:10])

inputs: [128000, 128000, 128006, 882, 128007, 271, 13347, 10896, 11, 12220]
labels: [128000, 128000, 128006, 78191, 128007, 271, 13347, 13, 1789, 4726]


# Create Dataloaders

In [10]:
# instantiate data collator
data_collator = DataCollatorWithPadding(tokenizer=pipeline.tokenizer)

# options
batch_size = 8

train_dataloader = DataLoader(tokenized_dataset['train'],
                              batch_size=batch_size, 
                              collate_fn=data_collator)

val_dataloader = DataLoader(tokenized_dataset['test'],
                            batch_size=batch_size,
                            collate_fn=data_collator)

In [11]:
# inspect sample batch
batch = next(iter(train_dataloader))
{key: val.shape for key, val in batch.items()}

{'input_ids': torch.Size([8, 1024]),
 'attention_mask': torch.Size([8, 1024]),
 'labels': torch.Size([8, 1024])}

In [12]:
outputs = pipeline.model(input_ids=batch['input_ids'],
                         attention_mask=batch['attention_mask'],
                         labels=batch['labels'])
print(outputs.loss)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


tensor(16.8531, grad_fn=<ToCopyBackward0>)


In [13]:
# test pre training
text = [{'role': 'system', 'content': 'you are a helpful medical chatbot'},
        {'role': 'user', 'content': 'I have a headache. What should I do?'}]
print(pipeline(text, max_length=100, truncation=True)[0]['generated_text'])

[{'role': 'system', 'content': 'you are a helpful medical chatbot'}, {'role': 'user', 'content': 'I have a headache. What should I do?'}, {'role': 'assistant', 'content': "Sorry to hear that you're experiencing a headache! There are several things you can try to help alleviate the discomfort. Here are some suggestions:\n\n1. **Stay hydrated**: Dehydration is a common cause of headaches. Drink a glass of water or other non-caffeinated fluid to see if that helps.\n2. **Take a pain relie"}]


# Training

In [14]:
# options
optimizer = AdamW(pipeline.model.parameters(), lr=1e-5)
num_epochs = 10
num_steps = num_epochs * len(train_dataloader)

# test after training
text = [{'role': 'system', 'content': 'You are a helpful medical chatbot'},
        {'role': 'user', 'content': 'I have a headache. What should I do?'}]

# loop
for epoch in range(num_epochs):
    
    print("=====================")
    print(f"Epoch {epoch + 1}")
    print("=====================")

    # set model to train mode
    pipeline.model.train()

    # initialize train loss, val loss
    running_train_loss = 0.0
    running_val_loss = 0.0

    # loop through train data
    print("Training...")
    i = 0
    for batch in tqdm(train_dataloader):

        # grab batch and map to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # forward pass
        outputs = pipeline.model(**batch)
        loss = outputs.loss
        print(f"batch loss: {loss:.4f}\r", end="")

        running_train_loss += loss.item()

        # backward pass
        loss.backward()

        # update optimizer
        optimizer.step()

        # zero gradients
        optimizer.zero_grad()
        
        i += 1

        # if i % 5 == 0:
        #     print(pipeline(text, max_length=100, truncation=True)[0]['generated_text'])
            
    # set model to eval mode
    pipeline.model.eval()

    for batch in val_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = pipeline.model(**batch)
            loss = outputs.loss
            running_val_loss += loss.item()
        
    val_loss = running_val_loss / len(val_dataloader)

    print("Printing example response...")
    print(pipeline(text, max_length=100, truncation=True)[0]['generated_text'])

    train_loss = running_train_loss / len(train_dataloader)
    print(f"Avg. Train Loss: {train_loss:.4f}, Avg. Val Loss: {val_loss:.4f}")
    # print("Evaluation metrics:", metric.compute())

print("Training Complete!")

Epoch 1
Training...


  0%|          | 0/563 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Prediction

In [15]:
# test after training
text = [{'role': 'system', 'content': 'You are a helpful medical chatbot'},
        {'role': 'user', 'content': 'I have a migraine. What should I do?'}]
print(pipeline(text, max_length=512, truncation=True)[0]['generated_text'])

[{'role': 'system', 'content': 'You are a helpful medical chatbot'}, {'role': 'user', 'content': 'I have a migraine. What should I do?'}, {'role': 'assistant', 'content': "Sorry to hear that you're experiencing a migraine! As a helpful medical chatbot, I'd like to offer some suggestions to help alleviate your symptoms.\n\n1. **Stay hydrated**: Dehydration is a common migraine trigger. Drink plenty of water or other non-caffeinated fluids to help replenish lost electrolytes.\n2. **Rest and relax**: Migraines can be triggered by stress, fatigue, and changes in sleep patterns. Take a break, lie down in a quiet, dark room, and try to relax.\n3. **Over-the-counter pain relief**: You can try taking over-the-counter pain relievers like ibuprofen (Advil, Motrin), acetaminophen (Tylenol), or aspirin. However, always follow the recommended dosage and consult your doctor if you have any concerns.\n4. **Cold or warm compresses**: Applying a cold or warm compress to the forehead, neck, or shoulders